In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize


# -------------------------------------------------------
# Paramètres physiques
# -------------------------------------------------------
f = 40000          # fréquence (Hz)
c0 = 343           # vitesse du son dans l'air (m/s)
ω = 2 * np.pi * f
k = ω / c0         # nombre d'onde

# ---- Constantes physiques ----
k = 2 * np.pi * 40000 / 343
R_p = 0.0005
c_p = 2400
rho_p = 28.59
c_0 = 343
rho_0 = 1.204
R_t = 0.005  # rayon transducteur

#---------------------------------------------------------
eps = 1e-4  # pas de dérivation

omega = 2 * np.pi * 40000
k1 = 4 * np.pi/3 * R_p**3 / 4 * (1/(c_0**2*rho_0) - 1/(c_p**2*rho_p))
k2 = 3*4*np.pi/3*R_p**3/4 * ((rho_0 - rho_p)/(omega**2 * rho_0 * (rho_0 + 2*rho_p)))

# -------------------------------------------------------
# Espace de simulation
# -------------------------------------------------------
N =8 # nombre de transducteur par axe (8x8)
a=0.01


# bornes symétriques : +/-5 cm latéralement, 8 cm en hauteur
Lx = 0.05
Ly = 0.05
DzT=0.08

x = np.linspace(-Lx, Lx, N)
y = np.linspace(-Ly, Ly, N)

X_T_b, Y_T_b = np.meshgrid(x, y)
Z_T_b=np.array([[0]*N]*N)

X_T_h, Y_T_h = np.meshgrid(x, y)
Z_T_h=np.array([[DzT]*N]*N)


##on passe tout à une dimension, dans le ref de la plaque :
X_T_b=X_T_b.flatten()
Y_T_b=Y_T_b.flatten()
Z_T_b=Z_T_b.flatten()

X_T_h=X_T_h.flatten()
Y_T_h=Y_T_h.flatten()
Z_T_h=Z_T_h.flatten()

X_T=np.concatenate((X_T_b,X_T_h))
Y_T=np.concatenate((Y_T_b,Y_T_h))
Z_T=np.concatenate((Z_T_b,Z_T_h))

In [2]:
 # === Fonctions utiles ===

def D_F_vectorized(xj, yj, zj, x, y, z):
    theta = np.arctan(np.sqrt((x - xj)**2 + (y - yj)**2) / np.where(np.abs(z - zj) < 1e-12, 1e-12, np.abs(z - zj)))
    df = np.sin(k * R_p * np.sin(theta)) / np.where(np.sin(k * R_p * np.sin(theta)) == 0, 1e-12, k * R_p * np.sin(theta))
    df = np.where(np.isnan(df), 1, df)
    return df

DJ={}
DF={}
def precompute_field_params(x, y, z):
    # Plaque du bas
    if (x,y,z) in DJ:
        return (DJ[(x,y,z)],DF[(x,y,z)])

    else :
      dx = X_T - x
      dy = Y_T - y
      dz = Z_T - z
      dj = np.sqrt(dx**2 + dy**2 + dz**2)
      df = D_F_vectorized(X_T, Y_T, Z_T, x, y, z)
      DJ[(x,y,z)]=dj
      DF[(x,y,z)]=df

      return (dj, df)

# === Calcul du champ total avec dj/df pré-calculés ===

def p_tot(phi_vals, x, y, z):
    dj, df = precompute_field_params(x, y, z)
    ptot = np.sum(df / dj * np.exp(1j * (k*dj + phi_vals)))
    return ptot

# === Gradient numérique ===

def grad_p(phi_vals, x, y, z):
    dp_dx = (p_tot(phi_vals, x+eps, y, z) - p_tot(phi_vals, x,y,z)) / eps
    dp_dy = (p_tot(phi_vals, x, y+eps, z) - p_tot(phi_vals, x,y,z)) / eps
    dp_dz = (p_tot(phi_vals, x, y, z+eps) - p_tot(phi_vals, x,y,z)) / eps
    return np.array([dp_dx, dp_dy, dp_dz])

# === Potentiel de Gorkov ===

def gorkov_potential(phi_vals, x, y, z):
    p_complex = p_tot(phi_vals, x,y,z)
    gradp = grad_p(phi_vals, x, y, z)

    return k1*abs(p_complex)**2 - k2*np.sum(np.abs(gradp)**2)

# === Laplacien du potentiel ===

def lap_U(phi_vals, x, y, z):
    U0 = gorkov_potential(phi_vals, x, y, z)
    Ux = gorkov_potential(phi_vals, x+eps, y, z)
    Uy = gorkov_potential(phi_vals, x, y+eps, z)
    Uz = gorkov_potential(phi_vals, x, y, z+eps)
    return (Ux - 2*U0 + gorkov_potential(phi_vals, x-eps, y, z))/eps**2 + \
           (Uy - 2*U0 + gorkov_potential(phi_vals, x, y-eps, z))/eps**2 + \
           (Uz - 2*U0 + gorkov_potential(phi_vals, x, y, z-eps))/eps**2

# === Fonction objectif pour l’optimisation ===

def F_opt(phi_vals, xf, yf, zf):
    ptot = p_tot(phi_vals, xf, yf, zf)
    lapU = lap_U(phi_vals, xf, yf, zf)
    return abs(ptot) - lapU

# === optimisation ===

def optimizer(xf, yf, zf):

    def objective(phi_vals):
        phi_mod = np.mod(phi_vals, 2*np.pi)
        return F_opt(phi_mod, xf, yf, zf)

    phi0 = np.zeros(2*(N**2))
    result = minimize(objective, phi0, method='L-BFGS-B')
    phases_mod = np.mod(result.x, 2*np.pi)

    print(", ".join(f"{a:.4f}" for a in phases_mod))
    return phases_mod

In [3]:
# === Point de focalisation ===
X_foc, Y_foc, Z_foc = 0, 0, 0.03

# === Lancement ===
optimizer(X_foc, Y_foc, Z_foc)

6.1687, 6.1484, 6.2429, 6.2162, 6.2162, 6.2429, 6.1484, 6.1687, 6.1484, 6.2162, 6.0998, 0.2206, 0.2206, 6.0998, 6.2162, 6.1484, 6.2429, 6.0998, 6.0707, 0.1673, 0.1673, 6.0707, 6.0998, 6.2429, 6.2162, 0.2206, 0.1673, 0.1772, 0.1772, 0.1673, 0.2206, 6.2162, 6.2162, 0.2206, 0.1673, 0.1772, 0.1772, 0.1673, 0.2206, 6.2162, 6.2429, 6.0998, 6.0707, 0.1673, 0.1673, 6.0707, 6.0998, 6.2429, 6.1484, 6.2162, 6.0998, 0.2206, 0.2206, 6.0998, 6.2162, 6.1484, 6.1687, 6.1484, 6.2429, 6.2162, 6.2162, 6.2429, 6.1484, 6.1687, 6.1754, 6.2464, 0.1211, 0.0048, 0.0048, 0.1211, 6.2464, 6.1754, 6.2464, 0.0048, 0.1503, 6.2116, 6.2116, 0.1503, 0.0048, 6.2464, 0.1211, 0.1503, 6.2482, 0.1306, 0.1306, 6.2482, 0.1503, 0.1211, 0.0048, 6.2116, 0.1306, 6.1161, 6.1161, 0.1306, 6.2116, 0.0048, 0.0048, 6.2116, 0.1306, 6.1161, 6.1161, 0.1306, 6.2116, 0.0048, 0.1211, 0.1503, 6.2482, 0.1306, 0.1306, 6.2482, 0.1503, 0.1211, 6.2464, 0.0048, 0.1503, 6.2116, 6.2116, 0.1503, 0.0048, 6.2464, 6.1754, 6.2464, 0.1211, 0.0048, 0.0048, 

array([6.16873840e+00, 6.14844241e+00, 6.24290127e+00, 6.21620977e+00,
       6.21620977e+00, 6.24290127e+00, 6.14844239e+00, 6.16873839e+00,
       6.14844240e+00, 6.21620977e+00, 6.09981677e+00, 2.20582586e-01,
       2.20582584e-01, 6.09981677e+00, 6.21620977e+00, 6.14844240e+00,
       6.24290127e+00, 6.09981677e+00, 6.07065441e+00, 1.67280590e-01,
       1.67280593e-01, 6.07065445e+00, 6.09981678e+00, 6.24290129e+00,
       6.21620978e+00, 2.20582606e-01, 1.67280589e-01, 1.77209901e-01,
       1.77209900e-01, 1.67280589e-01, 2.20582582e-01, 6.21620977e+00,
       6.21620978e+00, 2.20582606e-01, 1.67280589e-01, 1.77209899e-01,
       1.77209899e-01, 1.67280589e-01, 2.20582582e-01, 6.21620978e+00,
       6.24290127e+00, 6.09981677e+00, 6.07065445e+00, 1.67280590e-01,
       1.67280593e-01, 6.07065445e+00, 6.09981677e+00, 6.24290129e+00,
       6.14844239e+00, 6.21620977e+00, 6.09981677e+00, 2.20582582e-01,
       2.20582584e-01, 6.09981678e+00, 6.21620977e+00, 6.14844239e+00,
      